## Exercise 3
## MLP with TensorFlow 2
The objective of the exercise is to implement computational graphs in TensorFlow 2.0 to train and use such an architecture. The constraints we put ourselves is to use **low-level** functions of TensorFlow, i.e. we will not use high-level functions to compose layers and to train the parameters.

If you get this error in the execution of the first cell: ` ModuleNotFoundError: No module named 'tensorflow' `, it probably means TensorFlow 2.0 is not installed yet on your machine.

In [2]:
#############################
# MNIST Dataset Preparation #
#############################

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

mnist = tf.keras.datasets.mnist
(x_train, y_train_vec),(x_test, y_test_vec) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train_vec, 10, dtype='float64')
y_test = tf.keras.utils.to_categorical(y_test_vec, 10, dtype='float64')
N = x_train.shape[0]         # number of samples
D = x_train.shape[1]         # dimension of input sample
n_classes = y_train.shape[1] # output dim
print('MNIST data set ready. N={}, D={}, n_classes={}'.format(N,D,n_classes))

11493376/11490434 [==============================] - 2s 0us/step
MNIST data set ready. N=60000, D=784, n_classes=10


In [3]:
# Function to sample a random batch from dataset
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0,len(data))  # create an array of index values
    np.random.shuffle(idx)        # shuffle it
    idx = idx[:num]               # take the first n indexes = size of batch
    data_shuffle = data[idx]      # extract the batch using the random indexes
    labels_shuffle = labels[idx]  # extract the labels using the random indexes

    return data_shuffle, labels_shuffle

In [4]:
##################
# Training phase #
##################

E = 50                # number of epochs
B = 128               # batch size
N = x_train.shape[0]  # number of samples
D = x_train.shape[1]  # dimension of input sample
H = 600               # number of neurons
Amax = 5.0            # maximum learning rate alpha
Amin = 0.01           # minimum learning rate alpha

##############################################
#  COMPLETE CODE BELOW WHERE YOU SEE # ...   #
##############################################

# Build the computational graph
@tf.function # this decorator tells tf that a graph is defined
def mlp_train(
    x: tf.TensorSpec(shape=[None], dtype=tf.float64),
    y: tf.TensorSpec(shape=[None], dtype=tf.float64),
    alpha: tf.TensorSpec(shape=[None], dtype=tf.float64)
):
    # define nodes for forward computation for hidden neurons h and output neurons y_pred
    
    # h = ...  output of first layer after ReLu activation
    # h = tf.maximum(tf.matmul(x, w1) + b1, 0)
    h = tf.nn.relu(tf.matmul(x, w1) + b1)
    
    # y_pred = ... output of second layer after sigmoid activation
    # y_pred = 1 / (1 + tf.exp(-1 * (tf.matmul(h, w2) + b2, 0)))
    y_pred = tf.sigmoid(tf.matmul(h, w2) + b2)
    
    # define nodes for forward computation for hidden neurons h and output neurons y_pred
    diff = y_pred - y
    loss = tf.reduce_mean(tf.pow(diff, 2))
    
    # define the gradients
    grad_w1, grad_b1, grad_w2, grad_b2 = tf.gradients(loss, [w1, b1, w2, b2])
    
    # compute the new values of the gradients with the assign method (see slides)
    w1.assign(w1 - alpha * grad_w1)
    b1.assign(b1 - alpha * grad_b1)
    w2.assign(w2 - alpha * grad_w2)
    b2.assign(b2 - alpha * grad_b2)
    return y_pred, loss

# Init the tf.Variables w1, b1, w2, b2 following the given examples
w1 = tf.Variable(tf.random.truncated_normal((D, H), stddev = 0.1, dtype='float64'))
b1 = tf.Variable(tf.constant(0.0, shape=[H], dtype='float64'))
w2 = tf.Variable(tf.random.truncated_normal((H, 10), stddev = 0.1, dtype='float64'))
b2 = tf.Variable(tf.constant(0.0, shape=[10], dtype='float64'))

# Run the computational graph
J = [] # to store the evolution of loss J for each epoch
learning_rate = np.linspace(Amax, Amin, E)
start_time = time.time()
for epoch in range(E):
    J_epoch = 0.0
    for _ in range(int(N/B)): # number of batches to visit for 1 epoch
        # get batches calling the next_batch method provided above
        x_train_batch, y_train_batch = next_batch(B, x_train, y_train)
        with tf.device('/CPU:0'):  # change to /GPU:0 to move it to GPU
            # call the graph with the batched input, target and alpha A
            out = mlp_train(x_train_batch, y_train_batch, learning_rate[epoch])
        y_pred, loss_val = out
        J_epoch += loss_val
    J.append(J_epoch)
    print("epoch = {}, loss = {:.4f}".format(epoch, J_epoch))
print("Training took {:.2f}s".format(time.time()-start_time))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: name 'fscope' is not defined
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: name 'fscope' is not defined
epoch = 0, loss = 46.9905
epoch = 1, loss = 41.6208
epoch = 2, loss = 29.4246
epoch = 3, loss = 20.0393
epoch = 4, loss = 11.7576
epoch = 5, loss = 5.1989
epoch = 6, loss = 3.0188
epoch = 7, loss = 2.5270
epoch = 8, loss = 2.2359
epoch = 9, loss = 2.0152
epoch = 10, loss = 1.8419
epoch = 11, loss = 1.7283
epoch = 12, loss = 1.5201
epoch = 13, loss = 1.4105
epoch = 14, loss = 1.3244
epoch = 15, loss = 1.3459
epoch = 16, loss = 1.1903
epoch = 17, loss = 1.1618
epoch = 18, loss = 1.1366
epoch = 19, loss = 1.0532
epoch = 20, loss = 1.0225
epoch = 21, loss = 0.9344
epoch = 22, loss = 0.9072
epoch = 23, los

In [5]:
# Plot the evolution of the loss
plt.plot(J)

In [6]:
#################
# Testing phase #
#################

N = x_test.shape[0]  # number of samples
D = x_test.shape[1]  # dimension of input sample

##############################################
#  COMPLETE CODE BELOW WHERE YOU SEE # ...   #
##############################################
# Build the computational graph
@tf.function # this decorator tells tf that a graph is defined
def mlp_test(
    x: tf.TensorSpec(shape=[None], dtype=tf.float64),
    y: tf.TensorSpec(shape=[None], dtype=tf.float64)
):
    h = tf.nn.relu(tf.matmul(x, w1) + b1)
    y_pred = tf.sigmoid(tf.matmul(h, w2) + b2)
    return y_pred

# Run the computational graph
with tf.device('/CPU:0'):  # change to /GPU:0 to move it to GPU
    y_pred_test = mlp_test(x_test, y_test)

print('Forward pass on test set done.')
# At this stage, y_pred_test should contain the matrix of outputs on the test set with shape (N_test, 10)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: name 'fscope' is not defined
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: name 'fscope' is not defined
Forward pass on test set done.


In [7]:
# compute accuracy
y_winner = np.argmax(y_pred_test, axis=1)
N_test = y_winner.size
num_correct = (y_winner == y_test_vec).sum()
num_missed = N_test - num_correct
accuracy = num_correct * 1.0 / N_test
error_rate = num_missed * 1.0 / N_test
print('# samples  : ', N_test)
print('# correct  : ', num_correct)
print('# missed   : ', num_missed)
print('accuracy   :  %2.2f %%'% (accuracy*100.0))
print('error rate :  %2.2f %%'% (error_rate*100.0))

# samples  :  10000
# correct  :  9830
# missed   :  170
accuracy   :  98.30 %
error rate :  1.70 %


## Exercise 4
## MLP with TensorFlow 2 and Keras

### Sigmoid and MSE

In [14]:
# Direct from Slide 51

model = tf.keras.models.Sequential()

# Hidden layer - H neurons
model.add(tf.keras.layers.Dense(H, input_shape=(D,), use_bias=True, activation='relu', name="hidden-layer"))

# Output layer - #classes
model.add(tf.keras.layers.Dense(n_classes, input_shape=(H,), use_bias=True, activation='sigmoid', name="output-layer"))

model.summary()

sgd = tf.keras.optimizers.SGD(learning_rate=Amin)
model.compile(optimizer=sgd, loss='mse', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=B, epochs=E)

model.evaluate(x_test, y_test, verbose=2)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden-layer (Dense)         (None, 600)               471000    
_________________________________________________________________
output-layer (Dense)         (None, 10)                6010      
Total params: 477,010
Trainable params: 477,010
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 2s 40us/sample - loss: 0.1201 - accuracy: 0.2660
Epoch 2/50
60000/60000 [==============================] - 2s 36us/sample - loss: 0.0886 - accuracy: 0.4232
Epoch 3/50
60000/60000 [==============================] - 2s 35us/sample - loss: 0.0837 - accuracy: 0.4927
Epoch 4/50
60000/60000 [==============================] - 2s 36us/sample - loss: 0.0796 - accuracy: 0.5290
Epoch 5/50
60000/60000 [==============================] - 2

[0.026188663809001447, 0.8862]

### Softmax and CE

In [18]:
# Direct from Slide 51

model1 = tf.keras.models.Sequential()

# Hidden layer - H neurons
model1.add(tf.keras.layers.Dense(H, input_shape=(D,), use_bias=True, activation='relu', name="hidden-layer"))

# Output layer - #classes
model1.add(tf.keras.layers.Dense(n_classes, input_shape=(H,), use_bias=True, activation='softmax', name="output-layer"))

model1.summary()

model1.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

history1 = model1.fit(x_train, y_train, batch_size=B, epochs=E)

model1.evaluate(x_test, y_test, verbose=2)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden-layer (Dense)         (None, 600)               471000    
_________________________________________________________________
output-layer (Dense)         (None, 10)                6010      
Total params: 477,010
Trainable params: 477,010
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 2s 36us/sample - loss: 1.0753 - accuracy: 0.7731
Epoch 2/50
60000/60000 [==============================] - 2s 34us/sample - loss: 0.5134 - accuracy: 0.8743
Epoch 3/50
60000/60000 [==============================] - 2s 34us/sample - loss: 0.4147 - accuracy: 0.8907
Epoch 4/50
60000/60000 [==============================] - 2s 34us/sample - loss: 0.3696 - accuracy: 0.8990
Epoch 5/50
60000/60000 [==============================] - 

[0.12848244209513068, 0.9625]

## Summary

- From a mathematical point of view, computational graph breaks complex mathematics into simpler calculations. In NN, this helps us to modularize the calculations on each node, for both forward and backward passes. Each node only performs its own local computation and uses the calculated values from previous calculations or nodes.
- The performance of Tensorflow computational graph is equivalent if not slightly better than the performance of models built using Keras. Keras provides simple and intuitive ways to construct NN models, so we can focus more on tuning the models and their hyperparameters.